# !Getting started

## !!Starting Point: SparkSession

In [ ]:
from pyspark.sql import SparkSession

#### 在spark2.0以后，SparkSession创建SQLContext，并封装了sparkContext,  config就是sparkconf

In [ ]:
spark=SparkSession\
.builder.\
appName("python spark sql example")\
.config("spark.some.config.option","some-value")\
.getOrCreate()

## !!creating dataFrames

In [3]:
df=spark.read.json("/user/gongxf/spark/examples/src/main/resources/people.json")

In [4]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## !!Untyped Dataset Operations (aka DataFrame Operations)

In [ ]:
df.printSchema()

In [ ]:
df.select("name").show()

In [ ]:
df.select(df['name'],df['age']+1).show()

In [ ]:
df.filter(df['age']>21).show()

In [ ]:
df.groupBy("age").count().show()

## !!running sql Queries prgrammatically

In [ ]:
df.createOrReplaceTempView("people")

In [ ]:
sqlDF=spark.sql("select * from people")

In [ ]:
sqlDF.show()

## !!Global Temporary view

In [ ]:
df.createGlobalTempView("people")

In [ ]:
spark.sql("select * from global_temp.people").show()

In [ ]:
spark.newSession().sql("select * from global_temp.people").show()

## !! interoperating with RDDS

### !!! inferring the schema using reflection

In [ ]:
from pyspark.sql import Row

In [ ]:
sc=spark.sparkContext

In [ ]:
lines=sc.textFile("/user/gongxf/spark/examples/src/main/resources/people.txt")

In [ ]:
parts=lines.map(lambda l:l.split(","))

In [ ]:
people=parts.map(lambda p:Row(name=p[0],age=int(p[1])))

In [ ]:
schemaPeople=spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView("people")

In [ ]:
teenageres=spark.sql("select name from people where age>=13 and age<=19")

In [ ]:
teenageres.show()

In [ ]:
teenNames=teenageres.rdd.map(lambda p:"name"+p.name).collect()

In [ ]:
for name in teenNames:
    print(name)

### !!! programmatically specifying the schema

In [ ]:
from pyspark.sql.types import *

In [ ]:
sc=spark.sparkContext

In [ ]:
lines=sc.textFile("/user/gongxf/spark/examples/src/main/resources/people.txt")

In [ ]:
parts=lines.map(lambda l:l.split(","))
people=parts.map(lambda p:(p[0],p[1].strip()))

In [ ]:
schemaString="name age"

In [ ]:
fields=[StructField(field_name,StringType(),True) for field_name in schemaString.split()]

In [ ]:
schema=StructType(fields)

In [ ]:
schemaPeople=spark.createDataFrame(people,schema)

In [ ]:
schemaPeople.createOrReplaceTempView("people")

In [ ]:
results=spark.sql("select name from people")

In [ ]:
results.show()

# !Data sorces

## !! Generic load/save functions

In [ ]:
df=spark.read.load("/user/gongxf/spark/examples/src/main/resources/users.parquet")

In [ ]:
df.select("name","favorite_color").write.save("/user/gongxf/spark/examples/src/main/resources/namesAndFavColors.parquet")

### !!!Manually Specifying Options

In [ ]:
df=spark.read.load("/user/gongxf/spark/examples/src/main/resources/people.json"
                  ,format="json")

In [ ]:
df.select("name","age").write.save("namesAndAgess.parquet")

In [ ]:
df=spark.read.load("/user/gongxf/spark/examples/src/main/resources/people.csv"
                  ,format='csv',sep=";",inferSchema='true',header="true")

In [ ]:
df.show()

### !!!run sql on files directly

In [ ]:
df=spark.sql("select * from parquet.`/user/gongxf/namesAndAgess.parquet`")

In [ ]:
df.show()

### !!! save modes

### !!! Saving to Persistent tables

###  !!! bucketing,sorting and partitioning

In [ ]:
df.show()

In [ ]:
df.write.bucketBy(42,"name").sortBy("age").saveAsTable("people_bucketed1")

In [ ]:
df=spark.read.parquet("/user/gongxf/spark/examples/src/main/resources/users.parquet")

In [ ]:
df.write.partitionBy("favorite_color").bucketBy(42,"name")\
.saveAsTable("people_partitioned_bucketed")

## !!parquet files

### !!!loading data programmatically

In [ ]:
peopleDF=spark.read.json("/user/gongxf/spark/examples/src/main/resources/people.json")

In [ ]:
peopleDF.write.parquet("people.parquet")

In [ ]:
parquetFile=spark.read.parquet("people.parquet")

In [ ]:
parquetFile.createOrReplaceTempView("parquetFile")

In [ ]:
teenagers=spark.sql("select name from parquetFile where age>=12 and age<=19")

In [ ]:
teenageres.show()

### ！！！schema meerging

In [ ]:
from pyspark.sql import Row

In [ ]:
sc=spark.sparkContext

In [ ]:
squaresDF=spark.createDataFrame(sc.parallelize(range(1,6))\
            .map(lambda i:Row(single=i,double=i**2)))

In [ ]:
squaresDF.write.parquet("data/test_table/key=1")

In [ ]:
cubesDF=spark.createDataFrame(sc.parallelize(range(6,11))\
        .map(lambda i:Row(single=i,triple=i**3)))

In [ ]:
cubesDF.write.parquet("data/test_table/key=2")

In [ ]:
mergedDF=spark.read.option("mergeSchema","true").parquet("data/test_table")

In [ ]:
mergedDF.printSchema()

## Hive metastore Parquet table conversion

#### Metadata Refreshing

In [ ]:
# spark.catalog.refreshTable("aa")

### !!!json Datasets

In [ ]:
sc=spark.sparkContext

In [ ]:
path="/user/gongxf/spark/examples/src/main/resources/people.json"

In [ ]:
peopleDF=spark.read.json(path)

In [ ]:
peopleDF.printSchema()

In [ ]:
peopleDF.createOrReplaceTempView("people")

In [ ]:
teenagerNamesDF=spark.sql("select name from people where age between 13 and 19")

In [ ]:
teenagerNamesDF.show()

In [ ]:
jsonStrings=['{"name":"Yin","address":{"city":"columbus","state":"ohio"}}']

In [ ]:
otherPeopleRDD=sc.parallelize(jsonStrings)

In [ ]:
otherPeople=spark.read.json(otherPeopleRDD)

In [ ]:
otherPeople.show()

## !!hive table

In [ ]:
from os.path import expanduser,join,abspath

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [ ]:
warehouse_localtion=abspath('spark-warehouse')

In [ ]:
spark=SparkSession\
.builder\
.appName("python spark")\
.config("spark.sql.warehouse.dir",warehouse_localtion)\
.enableHiveSupport()\
.getOrCreate()

In [ ]:
spark.sql('create database if not exists kk')
spark.sql("use kk")

In [ ]:
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")

In [ ]:
spark.sql("load data local inpath 'data_examples/resources/kv1.txt' into table src")

In [ ]:
spark.sql("select * from src").show()

In [ ]:
spark.sql("select count(*) from src").show()

In [ ]:
sqlDF=spark.sql("select key,value from src where key<10 order by key")

In [ ]:
sqlDF.show()

In [ ]:
stringsDS=sqlDF.rdd.map(lambda row:"key:%d,Value:%s" %(row.key,row.value))

In [ ]:
for record in stringsDS.collect():
    print(record)

In [ ]:
Record=Row("key","value")

In [ ]:
recordsDF=spark.createDataFrame([])

In [ ]:
recordsDF = spark.createDataFrame([Record(i, "val_" + str(i)) for i in range(1, 101)])

In [ ]:
recordsDF.createOrReplaceTempView("records")

In [ ]:
spark.sql("SELECT * FROM records r JOIN src s ON r.key = s.key").show()

## !!PySpark Usage Guide for Pandas with Apache Arrow

### !!!Enabling for Conversion to/from Pandas

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")

In [ ]:
pdf=pd.DataFrame(np.random.rand(100,3))

In [ ]:
pdf.head(10)

In [ ]:
df=spark.createDataFrame(pdf)

In [ ]:
df.show()

In [ ]:
result_pdf=df.select("*").toPandas()

In [ ]:
result_pdf.head(3)

### !!!Pandas UDFs (a.k.a. Vectorized UDFs)

In [ ]:
import pandas as pd

In [ ]:
from pyspark.sql.functions import col,pandas_udf

In [ ]:
from pyspark.sql.types import LongType

In [ ]:
def multiply_func(a,b):
    return a*b

In [ ]:
multiply=pandas_udf(multiply_func,returnType=LongType())

In [ ]:
x=pd.Series([1,2,3])

In [ ]:
x

In [ ]:
print(multiply_func(x,x))

In [ ]:
df=spark.createDataFrame(pd.DataFrame(x,columns=["x"]))

In [ ]:
df.select(col("x")).show()

In [ ]:
df.select(multiply(col("x"),col("x"))).show()

## !!Grouped map

In [ ]:
from pyspark.sql.functions import pandas_udf,PandasUDFType

In [ ]:
df=spark.createDataFrame(\
    [(1,1.0),(1,2.0),(2,3.0),(2,5.0),(2,10.0)],("id","v")\
)

In [ ]:
@pandas_udf("id long,v double",PandasUDFType.GROUPED_MAP)
def substract_mean(pdf):
    v=pdf.v
    return pdf.assign(v=v-v.mean())

In [ ]:
df.groupby("id").apply(substract_mean).show()

In [ ]:
df.show()